In [54]:
# Using pandas & numpy
# get titanic dataframe

import pandas as pd
import numpy as np

PATH = "2017/Student Feedback Surveys-Superview.csv" 

df = pd.read_csv(PATH)

df.head(10)

,ID,Location,Track,Week,Rating (Num),Schedule Pacing
0,134,San Francisco,"Apps, Explorer",Week 1,3,Just right
1,36,Los Angeles,Apps,Week 1,4,A little too fast
2,117,San Francisco,Games,Week 1,4,Way too slow
3,253,NaN,NaN,Week 2,4,A little too fast
4,350,New York City,"Apps, Explorer",Week 1,4,Just right
5,23,Redwood City,Apps,Week 1,5,Just right
6,28,Los Angeles,Apps,Week 7,5,Just right
7,65,San Francisco,Apps,Week 1,5,A little too slow
8,101,Santa Clara,Apps,Week 1,5,A little too slow
9,124,Santa Clara,Apps,Week 1,5,Just right


## Data sanitisation for "Rating (Num)"

In [55]:
# Type of input rating?
df_test = df["Rating (Num)"][0]
print(type(df_test))


<class 'str'>


In [56]:
# bad data discovered!
ls_bad_data = df["Rating (Num)"].unique()

ls_bad_data

array(['3', '4', '5', '6', '7', '8', '9', '10', '0', '1', '2', '#ERROR!'],
      dtype=object)

In [57]:
# Fina all bad data from "Rating (Nums)" column
ARG_BAD_ENTRY = (df["Rating (Num)"] == "#ERROR!")

# Update dataframe to remove all bad entries in "Rating (Nums)" column
df = df[~ARG_BAD_ENTRY]

In [58]:
# overrite existing column (cast all Ratings to ints)
df["Rating (Num)"] = df["Rating (Num)"].astype(int)

for i in df["Rating (Num)"]:
    print(type(i))
    

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

- **Map**: Iterates over data and returns same-size data with function applied to each element.
    - EX: Map over data to multiply each entry in data by 5. 
- **Filter**: Iterates over data and returns subset of data with some filtering function applied to each element.
    - EX: Filter over data to return data within range (0, 10).
- **Reduce**: Iterates over data and returns single object with some accumulative reducing function applied to each element.
    - EX: Reduce all data to sum all entries in dataset.

In [ ]:
# change all elements of series/df column based on some arg --> BAD
# iterably filter elements based on logical expression/arg --> GOOD

## Create seperate data frames categorised by rating

In [59]:
# Add new column NPS_type for tag
def filter_nps(item):
    if item >= 0 and item < 6:
        return "Detractor"
    if item >= 6 and item < 8:
        return "Passive"
    if item >= 8 and item < 10:
        return "Promoter"

df["NPS_Type"] = df_ratings.apply(filter_nps)

In [48]:
df.head(5)

,ID,Location,Track,Week,Rating (Num),Schedule Pacing,NPS_Type
0,134,San Francisco,"Apps, Explorer",Week 1,3,Just right,Detractor
1,36,Los Angeles,Apps,Week 1,4,A little too fast,Detractor
2,117,San Francisco,Games,Week 1,4,Way too slow,Detractor
3,253,NaN,NaN,Week 2,4,A little too fast,Detractor
4,350,New York City,"Apps, Explorer",Week 1,4,Just right,Detractor


In [49]:
ARG_DETRACTORS = (df["NPS_Type"] == "Detractor")


df_detractors = df[ARG_DETRACTORS]


df_detractors.head(5)


,ID,Location,Track,Week,Rating (Num),Schedule Pacing,NPS_Type
0,134,San Francisco,"Apps, Explorer",Week 1,3,Just right,Detractor
1,36,Los Angeles,Apps,Week 1,4,A little too fast,Detractor
2,117,San Francisco,Games,Week 1,4,Way too slow,Detractor
3,253,NaN,NaN,Week 2,4,A little too fast,Detractor
4,350,New York City,"Apps, Explorer",Week 1,4,Just right,Detractor


In [50]:
ARG_PASSIVE = (df["NPS_Type"] == "Passive")


df_passive = df[ARG_PASSIVE]

df_passive.head(5)

,ID,Location,Track,Week,Rating (Num),Schedule Pacing,NPS_Type
20,37,Los Angeles,Games,Week 1,6,Just right,Passive
21,41,Chicago,"Apps, Explorer",Week 1,6,Just right,Passive
22,45,Chicago,Apps,Week 1,6,A little too slow,Passive
23,47,Chicago,Apps,Week 1,6,A little too slow,Passive
24,77,San Francisco,Apps,Week 1,6,A little too fast,Passive


In [60]:
ARG_PROMOTER = (df["NPS_Type"] == "Promoter")

df_promoter = df[ARG_PROMOTER]

df_promoter.head(5)

,ID,Location,Track,Week,Rating (Num),Schedule Pacing,NPS_Type
96,8,Atlanta,"Apps, Explorer",Week 1,8,Just right,Promoter
97,9,Atlanta,Apps,Week 1,8,Just right,Promoter
98,10,Atlanta,"Apps, Explorer",Week 1,8,Just right,Promoter
99,14,Atlanta,Apps,Week 1,8,A little too slow,Promoter
100,16,Redwood City,Apps,Week 1,8,A little too slow,Promoter


## Net Promoter Score 

In [64]:
# NPS = (Promoters - Detractors)  (Promoters + Passives + Detractors)

ARG_NPS =(len(df_promoter) - len(df_detractors)) / (len(df_detractors) + len(df_passive) + len(df_promoter))

NPS_SCORE = ARG_NPS * 100

NPS_SCORE

66.4804469273743